<a href="https://colab.research.google.com/github/hscrown/bigdata/blob/main/%EA%B8%B0%EC%B6%9C%EB%AC%B8%EC%A0%9C5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1유형

In [1]:
# 패키지 임포트
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 데이터 불러오기
df=pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/dataset/main/trash_bag.csv',encoding='cp949')
df

# 용도, 사용대상 항목 추출
print(df['용도'].unique())
print(df['사용대상'].unique())

# 음식물쓰레기와 가정용인 행만 남기기
df_home_trash=df[(df['용도']=='음식물쓰레기')&(df['사용대상']=='가정용')]
df_home_trash

# 2L 봉투를 사용하는 행만 남기기
result_df=df_home_trash[df_home_trash['2L가격']!=0]
result_df.head()

# 평균 구하기
result=int(result_df['2L가격'].mean())
print(result)

['생활쓰레기' '음식물쓰레기']
['기타' '가정용' '사업장용' '영업용' '영업장용' '범용']
119


In [2]:
# 패키지 임포트
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 데이터 불러오기
df=pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/dataset/main/BMI.csv')
df

# Height_m과 BMI 항목 추가
df["Height_m"]=df["Height"]/100
df["BMI"]=df["Weight"]/df["Height_m"]**2
df

# BMI 가 normal과 overweight인 구간의 인원 추출
normal=len(df[(df["BMI"]>=18.5)&(df["BMI"]<23)])
overweight=len(df[(df["BMI"]>=23)&(df["BMI"]<25)])
print(normal)
print(overweight)

# 인원 차이 구하기
result=abs(normal-overweight)
print(result)

47
19
28


In [3]:
# 패키지 임포트
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 데이터 불러오기
df=pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/dataset/main/students.csv',encoding='cp949')
df

# 순 전입학생 수 항목 추가
df['순 전입학생 수']=df['총 전입학생']-df['총 전출학생']
df.head()

# 학교별로 분류하기
result_df=df.groupby(['학교'])[['순 전입학생 수','전체학생 수']].sum()
print(result_df)

# 순 전입학생 수가 가장 많은 학교의 전체학생 수 구하기
result=result_df.loc[result_df['순 전입학생 수']==result_df['순 전입학생 수'].max(),'전체학생 수'].values[0]
print(result)

    순 전입학생 수  전체학생 수
학교                  
A         18     566
B         13     588
C         -9     528
D          7     548
E         14     603
F          1     481
G         10     618
H         -1     602
566


2유형

In [4]:
# 패키지 임포트
import pandas as pd

# 데이터 불러오기
X_test=pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/dataset/main/used_car_X_test.csv')
X_train=pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/dataset/main/used_car_X_train.csv')
y_train=pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/dataset/main/used_car_y_train.csv')
X_train

# 결측 여부 확인
print(f' X_train:{X_train.isnull().sum().sum()}')
print(f' y_train:{y_train.isnull().sum().sum()}')
print(f' X_test:{X_test.isnull().sum().sum()}')

# 범주형 데이터와 수치형 데이터로 나누기
X_train_word=X_train[['model','transmission','fuelType']]
X_test_word=X_test[['model','transmission','fuelType']]
print(X_train_word)
X_train_num=X_train.drop(columns=['id','model','transmission','fuelType'])
X_test_num=X_test.drop(columns=['id','model','transmission','fuelType'])
print(X_train_num)

# 데이터 스케일링 임포트
from sklearn.preprocessing import MinMaxScaler

# MinMax 스케일러 생성
scaler=MinMaxScaler()

# 해당 특성에 MinMax 스케일러를 적용하고 데이터 변환
X_train_num_scale=scaler.fit_transform(X_train_num)
X_test_num_scale=scaler.fit_transform(X_test_num)

# 데이터 프레임 정의
df_train_num_scale=pd.DataFrame(X_train_num_scale,columns=X_train_num.columns)
df_test_num_scale=pd.DataFrame(X_test_num_scale,columns=X_test_num.columns)

# 인코딩 시행
df_train_word=pd.get_dummies(X_train_word)
df_test_word=pd.get_dummies(X_test_word)

# 데이터프레임 컬럼 목록 설정
train_columns=set(df_train_word.columns)
test_columns=set(df_test_word.columns)

# 한쪽에만 있는 컬럼 출력
missing_in_test=train_columns-test_columns
missing_in_train=test_columns-train_columns
print("컬럼 목록 중 테스트 데이터에 없는 컬럼:",missing_in_test)
print("컬럼 목록 중 훈련 데이터에 없는 컬럼:",missing_in_train)

# 없는 컬럼을 0으로 채워줌
df_test_word['model_RS7']=0
df_test_word['model_S8']=0
df_test_word['model_A2']=0
df_test_word['model_S5']=0

# 두 데이터를 결합
df_train=pd.concat([df_train_num_scale,df_train_word],axis=1)
df_test=pd.concat([df_test_num_scale,df_test_word],axis=1)
print(df_train)

# 모델 학습 임포트
from pandas.core.common import random_state
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

# 모델 생성
model=xgb.XGBRegressor(random_state=77)

# train, validation 데이터 설정
X_train,X_val,y_train_t,y_val=train_test_split(df_train.values,y_train['price'].values,test_size=0.3)

# 모델 학습
model.fit(X_train,y_train_t)

# validation으로 성능 평가
y_pred=model.predict(X_val)
np.sqrt(mean_squared_error(y_val,y_pred))

# price 예측
y_pred=model.predict(df_test)
df=pd.DataFrame(X_test['id'])
df['price']=y_pred
print(df.head())

# 출력 형식 정하기
df.to_csv('result.csv',index=False)

 X_train:0
 y_train:0
 X_test:0
     model transmission fuelType
0       A1    Automatic   Petrol
1      RS4    Semi-Auto   Petrol
2       A8    Semi-Auto   Diesel
3       Q5    Semi-Auto   Diesel
4       A5    Semi-Auto   Diesel
...    ...          ...      ...
7463    A3       Manual   Petrol
7464    Q2       Manual   Diesel
7465    A1    Semi-Auto   Petrol
7466    Q3       Manual   Diesel
7467    Q7    Automatic   Diesel

[7468 rows x 3 columns]
      year  mileage  tax   mpg  engineSize
0     2019     3500  145  40.9         2.0
1     2020     2500  145  28.8         2.9
2     2019      500  145  40.4         3.0
3     2019     5089  150  38.2         2.0
4     2020     4951  145  51.4         2.0
...    ...      ...  ...   ...         ...
7463  2016    22633   30  58.9         1.4
7464  2017    13272  145  64.2         1.6
7465  2019     5000  145  40.9         2.0
7466  2019       10  145  42.8         2.0
7467  2012    91229  300  39.2         3.0

[7468 rows x 5 columns]
컬럼 목록 

3유형